In [218]:
!pip install opencv-python

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [219]:
from typing import List
import cv2 as cv
import numpy as np
import sys

In [233]:
class Face:
  """
  Class for holding the face data in colour, greyscale and the width and height.
  """
  def __init__(self, img: str):
    self.gImg = cv.imread(img, cv.IMREAD_GRAYSCALE)
    self.cImg = cv.imread(img, cv.IMREAD_COLOR)
    self.height, self.width = self.gImg.shape

class FaceFinder:
  """
  Face finding class. Takes Faces (needles) and an image (haystack).
  """
  def __init__(self, needles: List[Face], haystack: str):
    """Initialize the class"""
    self.needles = needles
    self.gray_haystack = cv.imread(haystack, cv.IMREAD_GRAYSCALE)
    self.col_haystack = cv.imread(haystack, cv.IMREAD_COLOR)
  def draw_rectangles(self, haystack: any, filter: List[np.ndarray]):
    # buffer = haystack
    """Draw rectangles around the faces"""
    for i, f in enumerate(filter):
        # Get points where threshold is met
        points = [(pt[0], pt[1]) for pt in zip(*f[::-1])]  # Swap coordinates and filter
        for pt in points:
            # Draw a green rectangle around each point
            cv.rectangle(haystack, pt, (pt[0] + self.needles[i].height, pt[1] + self.needles[i].width), (0, 255, 0, 255), 2)
    cv.imwrite("faces.png", haystack)

In [221]:
needles = [Face("glasses.png"), Face("glasses2.png"), Face("man.png"), Face("woman.png")]
ff = FaceFinder(needles, "crowd.png")

In [222]:
# Width and Height of the faces in the needles
print([f"Face: {(face.width, face.height)}" for face in ff.needles])
# Destructure width and Height of the haystack
(width, height) = ff.gray_haystack.shape
# for colour image:
# (width, height) = ff.col_haystack.shape[:-1]

['Face: (45, 68)', 'Face: (45, 68)', 'Face: (39, 56)', 'Face: (45, 46)']


In [223]:
results = [cv.matchTemplate(ff.col_haystack, face.cImg, cv.TM_CCOEFF)
          for face in ff.needles]

In [224]:
low = [np.where(result >= 0.4) for result in results]
mid = [np.where(result >= 0.6) for result in results]
high = [np.where(result >= 0.8) for result in results]


In [234]:
ff.draw_rectangles(ff.col_haystack, high)

In [226]:
# ff.draw_rectangles(ff.col_haystack, mid)
# cv.imwrite("faces_mid.jpg", ff.col_haystack)

In [227]:
# ff.draw_rectangles(ff.col_haystack, high)
# cv.imwrite("faces_high.jpg", ff.col_haystack)

In [231]:
img_rgb = cv.imread('crowd.png')
template = cv.imread('man.png')
w, h = template.shape[:-1]

res = cv.matchTemplate(img_rgb, template, cv.TM_CCOEFF_NORMED)
threshold = .8
loc = np.where(res >= threshold)
for pt in zip(*loc[::-1]):  # Switch columns and rows
    cv.rectangle(img_rgb, pt, (pt[0] + h, pt[1] + w), (0, 0, 255), 2)

cv.imwrite('result.png', img_rgb)

True